In [9]:
import tensorflow as tf
import pandas as pd

from pathlib import Path

### Using a simple RNN

In [10]:
path = Path("datasets/ridership/CTA_-_Ridership_-_Daily_Boarding_Totals.csv")
df = pd.read_csv(path, parse_dates=["service_date"])
# 将数据框的列名更改为较短的形式，以提高代码的可读性。
df.columns = ["date", "day_type", "bus", "rail", "total"]
# 根据日期列对数据框进行排序，并将日期列设置为数据框的索引。
df = df.sort_values("date").set_index("date")
# no need for total, it's just bus + rail
df = df.drop("total", axis=1)
# remove duplicated months (2011-10 and 2014-07)
df = df.drop_duplicates()

rail_train = df["rail"]["2016-01":"2018-12"] / 1e6
rail_valid = df["rail"]["2019-01":"2019-05"] / 1e6
rail_test = df["rail"]["2019-06"]

seq_length = 56
tf.random.set_seed(42)
train_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_train.to_numpy(),
    targets=rail_train[seq_length:],
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)
valid_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_valid.to_numpy(),
    targets=rail_valid[seq_length:],
    sequence_length=seq_length,
    batch_size=32
)

In [11]:
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

In [12]:
def fit_and_evaluate(model, train_set, valid_set, learning_rate, epochs=500):
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="val_mae", patience=50, restore_best_weights=True)
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=["mae"])
    history = model.fit(train_set, validation_data=valid_set, epochs=epochs, callbacks=[early_stopping_cb])
    valid_loss, valid_mae = model.evaluate(valid_set)
    return valid_mae * 1e6

In [13]:
fit_and_evaluate(model, train_ds, valid_ds, learning_rate=0.02)

Epoch 1/500
33/33 [==============================] - 2s 20ms/step - loss: 0.0300 - mae: 0.2049 - val_loss: 0.0118 - val_mae: 0.1152
Epoch 2/500
33/33 [==============================] - 0s 13ms/step - loss: 0.0117 - mae: 0.1188 - val_loss: 0.0120 - val_mae: 0.1140
Epoch 3/500
33/33 [==============================] - 0s 13ms/step - loss: 0.0115 - mae: 0.1193 - val_loss: 0.0116 - val_mae: 0.1142
Epoch 4/500
33/33 [==============================] - 0s 11ms/step - loss: 0.0114 - mae: 0.1203 - val_loss: 0.0115 - val_mae: 0.1140
Epoch 5/500
33/33 [==============================] - 0s 11ms/step - loss: 0.0113 - mae: 0.1200 - val_loss: 0.0114 - val_mae: 0.1137
Epoch 6/500
33/33 [==============================] - 0s 11ms/step - loss: 0.0112 - mae: 0.1201 - val_loss: 0.0114 - val_mae: 0.1134
Epoch 7/500
33/33 [==============================] - 0s 10ms/step - loss: 0.0112 - mae: 0.1196 - val_loss: 0.0113 - val_mae: 0.1131
Epoch 8/500
33/33 [==============================] - 0s 13ms/step - loss: 0.

102770.21676301956

In [ ]:
tf.random.set_seed(42)  # extra code – ensures reproducibility
univar_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, input_shape=[None, 1]),
    tf.keras.layers.Dense(1)  # no activation function by default
])

In [ ]:
# extra code – compiles, fits, and evaluates the model, like earlier
fit_and_evaluate(univar_model, train_ds, valid_ds, learning_rate=0.05)

### Deep RNNs

In [2]:
class LNSimpleRNNCell(tf.keras.layers.Layer):
    def __init__(self, units, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = tf.keras.layers.SimpleRNNCell(units, activation=None)
        self.layer_norm = tf.keras.layers.LayerNormalization()
        self.activation = tf.keras.activations.get(activation)
    
    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

In [3]:
tf.random.set_seed(42)
custom_ln_model = tf.keras.Sequential([
    tf.keras.layers.RNN(LNSimpleRNNCell(32), return_sequences=True, input_shape=[None, 5]),
    tf.keras.layers.Dense(14)
])

In [ ]:
fit_and_evaluate(custom_ln_model, seq2seq_train, seq2seq_valid,
                 learning_rate=0.1, epochs=5)